In [1]:
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-zh', trust_remote_code=True) # trust_remote_code is needed to use the encode method
embeddings = model.encode(['How is the weather today?', '今天天气怎么样?'])
print(cos_sim(embeddings[0], embeddings[1]))

/data/clipx/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


0.7860602


In [5]:
emoji_des_fn = "emoji_table.txt"
emoji_table = [l.strip().split("\t") for l in open(emoji_des_fn).readlines()]

In [7]:
emoji_des = {}
for d,k in emoji_table:
    if k in emoji_des:
        emoji_des[k].append(d)
    else:
        emoji_des[k] = [d]
emoji_des = {k:", ".join(v) for k,v in emoji_des.items()}

In [8]:
len(emoji_des)

1661

In [9]:
emoji_des

{'☑️': 'ballot box with check',
 '🌝': 'full moon with face, cheese, moon, smiling moon, phase, full, sky, moonface, night, werewolves, monsters, anthropomorphic, face, smiley moon, spooky',
 '🇧🇾': 'flag for belarus, regional indicator symbol letters by, belarusian flag',
 '🐩': 'vain, miniature poodle, dog, sophisticated, showy, clip, toy poodle, standard poodle, poodle',
 '✂️': 'black scissors, cutting',
 '⏸': 'double vertical bar',
 '🚴🏼': 'light brown bicyclist',
 '♏️': 'greek, stars, scorpius, sign, astrology, scorpio, scorpion, constellation, zodiac',
 '🚢': 'cruise, cruise ship, boat, ferry, ship',
 '🌽': 'husk, kernel, popcorn, stalk, cob, iowa, ear, food, yellow, maize, corn, ear of maize, corn on the cob',
 '🚂': 'steam locomotive, locomotive, steam train, railway locomotive, steam, train, engine',
 '🇮🇴': 'flag for british indian ocean territory, british indian ocean territory flag',
 '🅾️': 'negative squared latin capital letter o',
 '🛀🏼': 'light brown bath',
 '🇩🇰': 'flag for denma

In [10]:
emoji_embeddings = {k:model.encode(v) for k,v in emoji_des.items()}

In [11]:
len(emoji_embeddings)

1661

In [12]:
import pickle
# save emoji_embeddings
pickle.dump(emoji_embeddings, open('emoji_embeddings.pkl', 'wb'))

# load emoji_embeddings
emoji_embeddings = pickle.load(open('emoji_embeddings.pkl', 'rb'))

In [13]:
nomred_emoji_embeddings = {k:v/norm(v) for k,v in emoji_embeddings.items()}
pickle.dump(nomred_emoji_embeddings, open('emoji_normed_embeddings.pkl', 'wb'))
emoji_embeddings = pickle.load(open('emoji_normed_embeddings.pkl', 'rb'))

In [15]:
import numpy as np
emoji_list = np.array(list(emoji_embeddings.keys()))
# def decode_emoji(emoji_code):
#     return chr(int(emoji_code[2:], 16))
# emoji_list = np.array([decode_emoji(emoji_code) for emoji_code in emoji_code_list])

In [16]:
emoji_Fs = np.stack(list(emoji_embeddings.values()))

In [17]:
# dump emoji_Fs use numpy

np.save("emoji.npy", {"emoji_features": emoji_Fs, "emoji_list": emoji_list})
    


emoji = np.load('emoji.npy', allow_pickle=True).item()
emoji_features = emoji['emoji_features']


In [2]:
import numpy as np

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [75]:
def query_emoji(description, n_candidates=5):
    query = model.encode(description)
    query = query/norm(query)
    scores = emoji_Fs @ query
    inds = np.argpartition(-scores, min(n_candidates, len(emoji_list)-1))[:n_candidates]
    return emoji_list[inds].tolist(), emoji_code_list[inds].tolist(), scores[inds].tolist()

In [84]:
query_emoji("抽烟")

(['🌫', '🚬', '⛰', '🥓', '🕯'],
 ['U+1F32B', 'U+1F6AC', 'U+26F0', 'U+1F953', 'U+1F56F'],
 [0.34354108572006226,
  0.4129222631454468,
  0.3472767472267151,
  0.36155179142951965,
  0.3562557101249695])

In [82]:
np.argpartition([1,2],1)[:10]

array([0, 1])